<a href="https://colab.research.google.com/github/ismirnov56/MatmultCUDA/blob/main/1CPUPyCuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лаборататорная работа 

Умножение матриц на CPU и GPU.

Даны две матрицы A и B, размера N*N.
N от 100 до 2000. Измерение времени выполнения будем считать усреднённо по 10 попыткам.

# 1. CPU 

## 1.1 столбец на строку 
В силу хорошей оптимизации библиотеки numpy и слишком медленного вычисления на CPU итеративно, мною было приянто решение умножать строку на столбец и производить суммирование.

In [ ]:
import numpy as np
from time import time

def dot_(A,B):
    n = A.shape[0]
    C = np.zeros((n,n), dtype=np.float32)
    for i in range(0,n):
        for j in range(0,n):
                C[i,j] += np.sum(A[i,:] * B[:,j]) 
    return C

times_cpu_dot = []
for n in range(100, 2001, 100):
    A = np.random.randn(n, n).astype(np.float32)
    B = np.random.randn(n, n).astype(np.float32)
    t = 0.00
    for i in range(10):
        st = time()
        C = dot_(A, B)
        t += time() - st
    times_cpu_dot.append(t/10)

In [ ]:
times_cpu_dot

[0.057182598114013675,
 0.24844889640808104,
 0.569053339958191,
 1.0408404350280762,
 1.7208118200302125,
 2.545329236984253,
 3.6259246826171876,
 4.9408704996109005,
 6.345957517623901,
 8.273769760131836,
 9.732605457305908,
 11.850897836685181,
 14.464600157737731,
 16.618010640144348,
 19.606998634338378,
 22.407856917381288,
 25.54512333869934,
 27.676632714271545,
 30.627209424972534,
 34.31349744796753]

Небольшая проврерка работы

In [ ]:
n = 5
A = np.random.randn(n, n).astype(np.float32)
B = np.random.randn(n, n).astype(np.float32)
C1 = dot_(A, B)
C2 = np.dot(A, B)
print(n)
print('-'*80)
print(C1)
print('-'*80)
print(C2)

5
--------------------------------------------------------------------------------
[[ 0.8727852   2.2140727  -3.6167493  -3.503257   -1.45942   ]
 [ 3.6286452  -4.345679   -5.476016   -0.834162   -2.739592  ]
 [ 4.943053   -1.7179002  -7.0439863  -0.868645   -2.7627974 ]
 [ 2.935496   -1.6183989  -3.2860298  -1.5042415  -1.7602518 ]
 [ 0.44490707 -1.8233428   0.14871609  3.9935079   0.5605775 ]]
--------------------------------------------------------------------------------
[[ 0.8727852   2.214073   -3.6167493  -3.503257   -1.4594201 ]
 [ 3.6286452  -4.345679   -5.476016   -0.834162   -2.739592  ]
 [ 4.943053   -1.7179002  -7.0439863  -0.8686449  -2.7627976 ]
 [ 2.935496   -1.6183989  -3.28603    -1.5042413  -1.7602518 ]
 [ 0.44490704 -1.8233432   0.14871606  3.9935076   0.5605775 ]]


## 1.2 Numpy dot

In [ ]:
times_cpu_npdot = []
for n in range(100, 2001, 100):
    A = np.random.randn(n, n).astype(np.float32)
    B = np.random.randn(n, n).astype(np.float32)
    t = 0.00
    for i in range(10):
        st = time()
        C = np.dot(A, B)
        t += time() - st
    times_cpu_npdot.append(t/10)

In [ ]:
times_cpu_npdot

[0.0002835988998413086,
 0.0002912759780883789,
 0.0008139133453369141,
 0.0017032384872436523,
 0.0032474517822265623,
 0.0054214239120483395,
 0.008664226531982422,
 0.013333439826965332,
 0.01944878101348877,
 0.027039146423339842,
 0.03776769638061524,
 0.048171067237854005,
 0.06257970333099365,
 0.0789381980895996,
 0.10217525959014892,
 0.10167536735534669,
 0.12573959827423095,
 0.14981493949890137,
 0.18576388359069823,
 0.21355531215667725]

# 2. CUDA

Определим размерность блоков, пусть он будет равен для всех задач 32.

Размерность GRID будем вычислять как MATRIX_SIZE / BLOC_SIZE, если отстаток от деления не равен 0, то будем плюсовать 1. 

## 2.1 PyCUDA

Установим PyCuda

In [ ]:
!pip install pycuda

In [ ]:
import pycuda.autoinit
from pycuda.tools import make_default_context
make_default_context().get_device().name()

'Tesla T4'

### 2.1.1 Решаем задачу в 'лоб'.
Матрицы расположены в глобальной памяти.
По одной нити на каждый элемент произведения.

Проверка работы алгоритма в лоб.

In [ ]:
from pycuda import driver, compiler, gpuarray, tools
import pycuda.autoinit

BLOCK_SIZE = 32

kernel_code_template = """ 
__global__ void MatrixMulKernel(float *a, float *b, float *c) 
{
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int bx = blockIdx.x;
    int by = blockIdx.y;

    int row = by*blockDim.y + ty;
    int col = bx*blockDim.x + tx;

    if(row < %(MATRIX_SIZE)s && col < %(MATRIX_SIZE)s){
        float val = 0.0;
        for(int i=0; i<%(MATRIX_SIZE)s; ++i){
            val += a[row*%(MATRIX_SIZE)s + i]*b[%(MATRIX_SIZE)s*i + col];
        }
        c[row*%(MATRIX_SIZE)s + col] = val;
    }
}"""

MATRIX_SIZE = 10
if MATRIX_SIZE % BLOCK_SIZE != 0:
    GRID_SIZE = int(MATRIX_SIZE / BLOCK_SIZE) + 1 
else:
    GRID_SIZE = int(MATRIX_SIZE / BLOCK_SIZE)
a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
c_cpu = np.dot(a_cpu, b_cpu)

a_gpu = gpuarray.to_gpu(a_cpu)
b_gpu = gpuarray.to_gpu(b_cpu)

c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)
kernel_code = kernel_code_template % { 
            'MATRIX_SIZE': MATRIX_SIZE}

mod = compiler.SourceModule(kernel_code)

matrixmul = mod.get_function("MatrixMulKernel")

matrixmul(
      # inputs
      a_gpu, b_gpu,
      # output
      c_gpu,
      # grid of multiple blocks
      grid = (GRID_SIZE, GRID_SIZE, 1),
      # block of multiple threads
      block = (BLOCK_SIZE, BLOCK_SIZE, 1),
)
new_c = c_gpu.get()
print(MATRIX_SIZE)
print('-'*80)
print(new_c)
print('-'*80)
print(c_cpu)

10
--------------------------------------------------------------------------------
[[ 9.5178723e-02 -3.0408013e+00 -1.6342096e+00  3.8664782e+00
  -2.4041240e+00  7.6266899e+00  7.6707534e-02 -2.7174252e-01
  -4.1617970e+00 -4.3061137e+00]
 [ 2.4408448e+00  1.4186980e+00  2.2919505e+00 -1.3994753e+00
  -2.3594863e+00  1.0295769e+00  2.0540612e+00 -5.0135803e-01
   1.1066291e-01 -1.7442271e+00]
 [ 9.1642213e-01  4.1572517e-01  4.0755997e+00 -9.6282262e-01
   1.5606560e+00  6.1136141e+00 -1.0149521e-01 -4.9460311e+00
  -4.4111109e+00  2.7175677e+00]
 [ 4.6664634e-01  2.2396359e+00 -7.5080919e+00 -1.3747448e+00
  -1.5701208e-01  5.0243485e-01 -5.3436546e+00 -1.0699104e+00
   3.5423408e+00  8.8724762e-01]
 [ 1.2252010e+00  3.7863757e-03 -3.2837532e+00  2.9553261e+00
  -6.6604924e+00 -6.3805832e-03 -2.9731662e+00 -9.4810218e-01
  -4.2168722e-01  2.9286921e+00]
 [-1.8680295e+00 -2.0434134e+00 -4.2758889e+00  1.8256086e+00
   9.6860927e-01  1.8808706e+00 -4.0365648e+00  9.1430330e-01
   2.04

Расчёт времени.

In [ ]:
times_pycuda_glob = []
for n in range(100, 2001, 100):
    MATRIX_SIZE = n
    if MATRIX_SIZE % BLOCK_SIZE != 0:
        GRID_SIZE = int(MATRIX_SIZE / BLOCK_SIZE) + 1 
    else:
        GRID_SIZE = int(MATRIX_SIZE / BLOCK_SIZE)
    a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
    b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
    
    t = 0.00
    for i in range(10):
        st = time()
        a_gpu = gpuarray.to_gpu(a_cpu)
        b_gpu = gpuarray.to_gpu(b_cpu)

        c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)
        kernel_code = kernel_code_template % { 
            'MATRIX_SIZE': MATRIX_SIZE}

        mod = compiler.SourceModule(kernel_code)

        matrixmul = mod.get_function("MatrixMulKernel")

        matrixmul(
            # inputs
            a_gpu, b_gpu,
            # output
            c_gpu,
            # grid of multiple blocks
            grid = (GRID_SIZE, GRID_SIZE, 1),
            # block of multiple threads
            block = (BLOCK_SIZE, BLOCK_SIZE, 1),
        )
        new_c = c_gpu.get()
        t += time() - st
    times_pycuda_glob.append(t/10)

In [ ]:
times_pycuda_glob

[0.0276444673538208,
 0.02657451629638672,
 0.028185415267944335,
 0.027504897117614745,
 0.02899048328399658,
 0.030380797386169434,
 0.030751371383666994,
 0.032529616355896,
 0.035014891624450685,
 0.036918258666992186,
 0.041953206062316895,
 0.04360098838806152,
 0.04504399299621582,
 0.04587640762329102,
 0.04921045303344727,
 0.04610986709594726,
 0.049653244018554685,
 0.055193972587585446,
 0.05788938999176026,
 0.06183755397796631]

На каждый элемент произведения матриц
2*N арифметических операций
2*N обращений к глобальной памяти
Таким образом можно сделать вывод, что быстродействие алгоритма завст от доступа к памяти, что является более долгим.

## 2.1.2 Решение с использованием разделяемой памяти

При вычислении C постоянно
используются одни и те же
элементы из A и B

Эти элементы образуют полосы, Размер одной полосы N на BLOCK_SIZE, поэтому даже одна полоса в разделяемую память не помещается. Для этого будет производится, делени на блоки BLOCK_SIZE на BLOCK_SIZE. Хранить в shared-памяти необходимо только две подматрицы BLOCK_SIZE на BLOCK_SIZE.

Загрузить необходимые данные (часть) в
разделяемую память (из глобальной)
__syncthreads();
Выполнить вычисления над обработанными данными
__syncthreads();
Записать результат в глобальную память



Также добавим проверку, на то, что матрица не кратная. Если это так, то заполняем shared блоки 0

In [ ]:
from pycuda import driver, compiler, gpuarray, tools
import pycuda.autoinit

BLOCK_SIZE = 32

kernel_code_template_shared = """ 
__global__ void MatrixMulKernelShared(float *a, float *b, float *c) 
{
    __shared__ float sA[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s]; 
    __shared__ float sB[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];

    int row = blockDim.y*blockIdx.y + threadIdx.y;
    int col = blockDim.x*blockIdx.x + threadIdx.x;
    float value = 0.0;

    sA[threadIdx.y][threadIdx.x] = 0.0;
    sB[threadIdx.y][threadIdx.x] = 0.0;

    for(int i = 0; i < (((%(MATRIX_SIZE)s - 1)/%(BLOCK_SIZE)s) + 1); i++)
    {
        if((row < %(MATRIX_SIZE)s) && (threadIdx.x + (i* %(BLOCK_SIZE)s)) < %(MATRIX_SIZE)s)
            sA[threadIdx.y][threadIdx.x] = a[(row*%(MATRIX_SIZE)s) + threadIdx.x + (i* %(BLOCK_SIZE)s)];
        if(col < %(MATRIX_SIZE)s && (threadIdx.y + i * %(BLOCK_SIZE)s) < %(MATRIX_SIZE)s)
            sB[threadIdx.y][threadIdx.x] = b[(threadIdx.y + i * %(BLOCK_SIZE)s)*%(MATRIX_SIZE)s + col];
        __syncthreads();

        for(int j = 0; j <  %(BLOCK_SIZE)s; ++j)
            value += sA[threadIdx.y][j] * sB[j][threadIdx.x];
    }
    if(row < %(MATRIX_SIZE)s && col < %(MATRIX_SIZE)s)
        c[row*%(MATRIX_SIZE)s + col] = value;

}"""

MATRIX_SIZE = 5
if MATRIX_SIZE % BLOCK_SIZE != 0:
    GRID_SIZE = int(MATRIX_SIZE / BLOCK_SIZE) + 1 
else:
    GRID_SIZE = int(MATRIX_SIZE / BLOCK_SIZE)
a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
c_cpu = np.dot(a_cpu, b_cpu)

a_gpu = gpuarray.to_gpu(a_cpu)
b_gpu = gpuarray.to_gpu(b_cpu)

c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)

kernel_code_shared = kernel_code_template_shared % {
            'BLOCK_SIZE': BLOCK_SIZE,
            'MATRIX_SIZE': MATRIX_SIZE}

mod = compiler.SourceModule(kernel_code_shared)

matrixmul = mod.get_function("MatrixMulKernelShared")

matrixmul(
      # inputs
      a_gpu, b_gpu,
      # output
      c_gpu,
      # grid of multiple blocks
      grid = (GRID_SIZE, GRID_SIZE, 1),
      # block of multiple threads
      block = (BLOCK_SIZE, BLOCK_SIZE, 1),
)
new_c = c_gpu.get()
print(MATRIX_SIZE)
print('-'*80)
print(new_c)
print('-'*80)
print(c_cpu)

5
--------------------------------------------------------------------------------
[[ 1.1870606e-03  6.7894685e-01 -1.4249148e+00  4.3021947e-01
  -6.6550052e-01]
 [ 1.3419559e+00 -1.9053680e+00  3.1952651e+00 -2.0126512e+00
   7.8941145e+00]
 [ 5.3654552e-01 -6.1682677e-01  3.3089730e-01 -7.0139545e-01
  -3.1006208e-01]
 [-1.1755749e+00 -1.3618978e+00 -9.6797323e-01 -2.2385228e+00
   1.4731802e+00]
 [-1.6252077e-01  7.9803962e-01  5.2018631e-01  6.6955522e-02
   5.7431364e-01]]
--------------------------------------------------------------------------------
[[ 1.1870606e-03  6.7894685e-01 -1.4249148e+00  4.3021947e-01
  -6.6550052e-01]
 [ 1.3419559e+00 -1.9053680e+00  3.1952651e+00 -2.0126512e+00
   7.8941145e+00]
 [ 5.3654552e-01 -6.1682677e-01  3.3089730e-01 -7.0139545e-01
  -3.1006208e-01]
 [-1.1755749e+00 -1.3618978e+00 -9.6797323e-01 -2.2385228e+00
   1.4731802e+00]
 [-1.6252077e-01  7.9803962e-01  5.2018631e-01  6.6955522e-02
   5.7431364e-01]]


Алгоритм работает произведём измерение времени

In [ ]:
times_pycuda_shared = []
for n in range(100, 2001, 100):
    MATRIX_SIZE = n
    if MATRIX_SIZE % BLOCK_SIZE != 0:
        GRID_SIZE = int(MATRIX_SIZE / BLOCK_SIZE) + 1 
    else:
        GRID_SIZE = int(MATRIX_SIZE / BLOCK_SIZE)
    a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
    b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
    
    t = 0.00
    for i in range(10):
        st = time()
        a_gpu = gpuarray.to_gpu(a_cpu)
        b_gpu = gpuarray.to_gpu(b_cpu)

        c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)
        kernel_code_shared = kernel_code_template_shared % {
            'BLOCK_SIZE': BLOCK_SIZE,
            'MATRIX_SIZE': MATRIX_SIZE}

        mod = compiler.SourceModule(kernel_code_shared)

        matrixmul = mod.get_function("MatrixMulKernelShared")

        matrixmul(
            # inputs
            a_gpu, b_gpu,
            # output
            c_gpu,
            # grid of multiple blocks
            grid = (GRID_SIZE, GRID_SIZE, 1),
            # block of multiple threads
            block = (BLOCK_SIZE, BLOCK_SIZE, 1),
        )
        new_c = c_gpu.get()
        t += time() - st
    times_pycuda_shared.append(t/10)

In [ ]:
times_pycuda_shared

[0.028669977188110353,
 0.02777731418609619,
 0.02806820869445801,
 0.028605055809020997,
 0.030236291885375976,
 0.030353856086730958,
 0.0315310001373291,
 0.03389830589294433,
 0.036460375785827635,
 0.0373795747756958,
 0.039292764663696286,
 0.04307260513305664,
 0.04505701065063476,
 0.04582734107971191,
 0.04802858829498291,
 0.049235296249389646,
 0.04838404655456543,
 0.052605485916137694,
 0.056986427307128905,
 0.05988717079162598]

## 2.2 Numba

Представлен во другом notebook, а резуьтаты сохраним с помощью pandas.

In [ ]:
import pandas as pd

data = {
    'N': [*range(100, 2001, 100)],
    'cpu': times_cpu_dot,
    'numpy': times_cpu_npdot,
    'pycuda': times_pycuda_glob,
    'pycuda with shared': times_pycuda_shared
}

df = pd.DataFrame(data)

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df.to_csv('drive/My Drive/data/pycuda.csv', index=False)